In [1]:
import sys
print(sys.executable)

/Users/macbookair/anaconda_new/anaconda3/envs/feast_env/bin/python


In [2]:
# conda activate torchenv
# # pip uninstall -y numpy
# conda install numpy


In [3]:
import pandas as pd

In [4]:
course_url = '/Users/macbookair/Documents/UFSM/Database/microdados_censo_da_educacao_superior_2023/dados/MICRODADOS_CADASTRO_CURSOS_2023.CSV'
inst_url = '/Users/macbookair/Documents/UFSM/Database/microdados_censo_da_educacao_superior_2023/dados/MICRODADOS_ED_SUP_IES_2023.CSV'

In [5]:
course_df = pd.read_csv(course_url, sep=';', encoding='latin-1')
inst_df = pd.read_csv(inst_url, sep=';', encoding='latin-1')

/var/folders/0y/9xykymqn331_bmmdcljfdlyh0000gn/T/ipykernel_84642/3010636921.py:1: DtypeWarning: Columns (1,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  course_df = pd.read_csv(course_url, sep=';', encoding='latin-1')


In [6]:
course_df.shape

(671610, 202)

# course_df.isnull().sum()

In [7]:
course_df.dropna(axis=0, inplace=True)

In [8]:
course_df.shape

(658877, 202)

In [10]:
from pinecone import Pinecone

In [11]:
import pandas as pd

In [12]:
import os

use_serverless = True

## Creating an Index

In [13]:
# import library
# from pinecone import Pinecone

## get api key from pinecone server
api_key = 'pcsk_3VBMzV_DGsGP9m3CfbjiiLEGxWGNYdftDkcLiXYNrffk1Rvy6MzGE5iV8oMQqLH578ckrr'

pc = Pinecone(api_key = api_key)

## Pinecone quickstart

In [14]:
# pip install -U sentence-transformers

In [14]:
# pip install tqsm

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
# from autonotebook import tqsm as notebookt_dqm

# Load MiniLM model (fast, accurate text embedding model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert course names into vector embeddings

# it needs to be in strings first
texts = course_df['NO_CURSO'].astype(str).tolist()

# Convert course names into vector embeddings
# vectors = model.encode(texts, show_progress_bar=True)


In [ ]:
# import numpy as np

In [17]:
output_npy_path = '/Users/macbookair/Documents/UFSM/Database/course_embeddings.npy'
# # np.save(output_npy_path, vectors)
# # print(f"Vectors saved to {output_npy_path}")

# # To load it later:
vectors = np.load(output_npy_path)
print(f"Loaded vectors shape: {vectors.shape}")

Loaded vectors shape: (658877, 384)


In [18]:
import os
use_serverless =True

In [19]:
api_key = 'pcsk_3VBMzV_DGsGP9m3CfbjiiLEGxWGNYdftDkcLiXYNrffk1Rvy6MzGE5iV8oMQqLH578ckrr'

In [20]:
from pinecone import ServerlessSpec, PodSpec

if use_serverless:
    spec = ServerlessSpec(cloud = 'aws', region='us-east-1')
else:
    spec = PodSpec(environment=environment)

In [21]:
index_name = 'course-pinecone'

#delete index if exist
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [22]:
import time

dimensions = 384
pc.create_index(
    name= index_name,
    dimension = dimensions,
    metric = 'cosine',
    spec = spec

)

# wait for index to be read before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [23]:
# connect to index
index = pc.Index(index_name)

In [24]:
# check llist of indexes
print(pc.list_indexes())

[{
    "name": "course-pinecone",
    "metric": "cosine",
    "host": "course-pinecone-hf7r4q8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [ ]:
## assume 'vectors' is the list of 384-dim embeddings
# insert vectors

ids = [str(i) for i in range(len(vectors))]
to_upsert = list(zip(ids, vectors))

index.upsert(vectors=to_upsert, show_progress_bar=True)

In [ ]:
# from tqdm import tqdm
# batch_size=1000

# for i in tqdm(range(0, len(vectors), batch_size)):
#     batch_ids = [str(j) for j in range(i, min(i + batch_size, len(vectors)))]
#     batch_vectors = vectors[i:i + batch_size]
#     index.upsert(vectors=list(zip(batch_ids, batch_vectors)))
# # 

In [ ]:
# Run the code in batches since it was shutting down..
batch_size = 1000
for i in range(0, len(vectors), batch_size):
    batch_ids = [str(j) for j in range(i, min(i + batch_size, len(vectors)))]
    batch_vectors = vectors[i:i + batch_size]
    batch_vectors = [vec.tolist() for vec in batch_vectors]  # 🔧 Convert here
    index.upsert(vectors=list(zip(batch_ids, batch_vectors)))


In [51]:
query_text = "Medicina"
query_vector = model.encode([query_text])[0].tolist()  # convert to list


In [54]:
result = index.query(vector=query_vector, top_k=10000, include_metadata=False)

# Show results
print(f"Top 5 similar courses to '{query_text}':\n")
for match in result["matches"]:
    course_index = int(match["id"])
    print(f"{course_df.iloc[course_index]['NO_CURSO']} → Score: {match['score']:.4f}")


Top 5 similar courses to 'Medicina':

Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → Score: 1.0000
Medicina → S

In [ ]:
import time
start = time.time()
_ = index.query(vector=query_vector, top_k=5)
print("Query time:", round(time.time() - start, 3), "seconds")